In [1]:
## table 만들기

# 기간(month_year) table
create_MY = """
create table MY(
  MY_id  varchar(20)  primary key,
  year varchar(20),
  month varchar(20)
)
"""
# 정확하지 않다. 

# type table
create_type = """
create table type(
  type_id  int  auto_increment  primary key,
  type_name  varchar(20) not null unique
)
"""

# Purpose of Car = PC table 
create_PC = """
create table PC(
  PC_id  int  auto_increment  primary key,
  PC_name  varchar(20) not null unique
)
"""

# gu table 
create_gu = """
create table gu(
  gu_id int auto_increment  primary key,
  gu_name varchar(20) unique
)
"""

# 자동차 등록 대수(main)
create_number = """
create table number(
  number_id  int  auto_increment  primary key,
  gu_name  varchar(20),
  MY_id  varchar(20), 
  type_id int,
  PC_id int,
  the_number_of_registered_vehicle int,

  CONSTRAINT fk_number_gu FOREIGN KEY (gu_name) REFERENCES gu(gu_name) ON DELETE SET NULL,
  CONSTRAINT fk_number_MY FOREIGN KEY (MY_id) REFERENCES MY(MY_id) ON DELETE SET NULL,
  CONSTRAINT fk_number_type FOREIGN KEY (type_id) REFERENCES type(type_id) ON DELETE SET NULL,
  CONSTRAINT fk_number_usage FOREIGN KEY (PC_id) REFERENCES PC(PC_id) ON DELETE SET NULL
)
"""

In [2]:
import pymysql
import configparser as parser

# database의 내용을 parser로 읽어와서 코드에서는 정보가 드러나지 않도록 하고 싶습니다. 
# 그에 따른 조치사항으로 parser를 import 했습니다.  

try:
    conn = None
    
    props = parser.ConfigParser()  # parser 생성
    props.read("C:/Users/Playdata/Desktop/SK AI camp_6기_조하늘/SKN06-1st-7Team/config.ini.ini")     # ini 파일 읽기 / 절대 경로로 변환
    config = props['skn06-1st-7team']        # section이름으로 읽기
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host=config['host'],    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user=config['user'],        # username: str
        password=config['password'],    # password: str
        db=config['db']            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. table 생성
    cursor.execute("drop table if exists number") 
    cursor.execute("drop table if exists gu") 
    cursor.execute("drop table if exists MY") 
    cursor.execute("drop table if exists type") 
    cursor.execute("drop table if exists PC") 
    cursor.execute(create_MY)
    cursor.execute(create_type)
    cursor.execute(create_PC)
    cursor.execute(create_gu)
    cursor.execute(create_number)
    
finally:
    # 4. 연결닫기(끊기)
    if conn: # conn != None
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

In [3]:
## csv 파일 읽기
import csv

with open('자동차현황_변형.csv', 'r', encoding='ANSI') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)

# data는 이제 CSV 파일의 내용을 담은 리스트입니다.


In [4]:
# data[0],data[1]은 열의 이름을 나타내고 data[2]부터 지역별 등록 대수를 나타냄
len(data) # 6708 x 22 행

6706

In [5]:
# MY table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# create_MY = """
# create table MY(
#   MY_id  varchar(20)  primary key,
#   year varchar(20),
#   month varchar(20)
# )
# """

i = 0
data_MY = [] # sql에 최종적으로 넣을 값. 중복 제거 


for i in range(6706): # 자동차현황_변형은 총 6706개의 행을 가지고 있다. /gu_table
    if data[i][1] == "서울":
        # data 분류 필요
        insert_list = [] # 변환되는 값.
        year:str = None
        month:str = None
        insert_list.append(data[i][0]) # 월
        year = insert_list[0][:4]
        month = insert_list[0][-2:]
        insert_list.append(year)
        insert_list.append(month)
        data_MY.append(list(insert_list))
        

# 중복 제거
result_MY = []
for value in data_MY:
    if value not in result_MY:
        result_MY.append(value)


 # insert 하기
insert_sql = """insert into MY(MY_id, year, month) values(%s, %s, %s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, result_MY)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

insert된 총 행수: 25


In [6]:
# type table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# create table type(
#   type_id  int  auto_increment  primary key,
#   type_name  varchar(20) not null unique
# )
# """

data_type = ["승용", "승합", "화물", "특수"]
 # sql에 최종적으로 넣을 값
 # insert 하기
insert_sql = """insert into type(type_name) values(%s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, data_type)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

insert된 총 행수: 4


In [7]:
# PC table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# create table PC(
#   PC_id  int  auto_increment  primary key,
#   PC_name  varchar(20) not null unique
# )
# """

data_PC = ["관용", "자가용", "영업용"]
 # sql에 최종적으로 넣을 값
 # insert 하기
insert_sql = """insert into PC(PC_name) values(%s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, data_PC)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

insert된 총 행수: 3


In [8]:
# gu table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# create table gu(
#   gu_id  varchar(20)  primary key)

data_gu = [] # sql에 최종적으로 넣을 값.


for i in range(6706): # 자동차현황_변형은 총 6706개의 행을 가지고 있다. /gu_table
    if data[i][1] == "서울":
        # data 분류 필요
        data_gu.append(data[i][2])

result_gu = []
# data 중복되는 값 제거 
for values in data_gu:
    if values not in result_gu:
        result_gu.append(values)


# '계'로 삽입된 행 제거 
result_gu.pop() # 가장 마지막에 있는 행으로 제거. 

# insert 하기
insert_sql = """insert into gu(gu_name) values(%s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, result_gu)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

insert된 총 행수: 25


In [25]:
# number table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# create_number = """
# create table number(
#   number_id  int  auto_increment  primary key,
#   gu_name  varchar(20),
#   MY_id  varchar(20), 
#   type_id int,
#   PC_id int,
#   the_number_of_registered_vehicle int,

#   CONSTRAINT fk_number_gu FOREIGN KEY (gu_name) REFERENCES gu(gu_name) ON DELETE SET NULL,
#   CONSTRAINT fk_number_MY FOREIGN KEY (MY_id) REFERENCES MY(MY_id) ON DELETE SET NULL,
#   CONSTRAINT fk_number_type FOREIGN KEY (type_id) REFERENCES type(type_id) ON DELETE SET NULL,
#   CONSTRAINT fk_number_usage FOREIGN KEY (PC_id) REFERENCES PC(PC_id) ON DELETE SET NULL
# )
# """
data_number = [] # sql에 최종적으로 넣을 값.


for i in range(len(data)): # 자동차현황_변형은 총 6706개의 행을 가지고 있다. /gu_table
    if data[i][1] == "서울":
        # data 분류 필요
        insert_list = [] # 변환되는 값.
        if data[i][2] == "계":
            continue  # 실행되면 안 되기 때문에 continue로 다음 iteration으로 넘긴다. 
        else:
            insert_list.append(data[i][2]) # 구 이름 삽입 및 구 이름이 '계'로 된 행은 받지 않는다. 
            insert_list.append(data[i][0]) # 월
        for j in range(4): # 차종에 따른 분류, 1:승용, 2:승합, 3:화물, 4:수수
            if len(insert_list) > 2:
                insert_list.pop()
                insert_list.pop()
                insert_list.pop()
                insert_list.append(j+1) # 차종 입력
            else: # 첫시행
                insert_list.append(j+1) # 차종 입력
                            
            for v in range(3): # 용도 분류, 1:관용, 2:자가용, 3: 영업용
                if len(insert_list) > 3:# 4,5번 값이 존재할 경우
                    insert_list.pop()
                    insert_list.pop()# 2개의 값을 지워준다.
                    insert_list.append(v+1) # 용도 입력
                    insert_list.append(int(data[i][4*j + 3 + v])) # data에서 조건에 맞는 차 넘버수 불러오기
                    data_number.append(list(insert_list)) # data_handled 에 넣기 
                else: # 첫시행
                    insert_list.append(v+1) # 용도 입력
                    insert_list.append(int(data[i][4*j + 3 + v])) # data에서 조건에 맞는 차 넘버수 불러오기
                    data_number.append(list(insert_list))
    else: 
        pass

len(data_number)

# insert 하기
insert_sql = """insert into number(gu_name, MY_id, type_id, PC_id, the_number_of_registered_vehicle) 
            values(%s, %s, %s, %s, %s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db'],
                     max_allowed_packet=67108864000  # 64MB
                     ) as conn:
    with conn.cursor() as cursor:
        batch_size = 500
        for i in range(0, len(data_number), batch_size):
            batch = data_number[i:i+batch_size]
            cursor.executemany(insert_sql, batch)
            conn.commit()
        # cnt = cursor.executemany(insert_sql, data_number)  # executemany() 결과 행수 반환.
        # conn.commit()
        

# print("insert된 총 행수:", cnt)
print("실행 완료") # 1000까지 입력이 되었다. 

실행 완료


In [13]:
len(data_number) # 7500개가 있으니 data_number list를 8개로 나누자. 단위는 1000

7500

In [21]:
# 1000-1999
data_number_2 = []
data_number_2 = data_number[1000:2000]
print(data_number_2[:5])
print("-"*50)
print(data_number[999])
print("-"*50)
print(data_number[1000])

# list insert for data_number_2
# insert 하기
insert_sql = """insert into number(gu_name, MY_id, type_id, PC_id, the_number_of_registered_vehicle) 
            values(%s, %s, %s, %s, %s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db'],
                     ) as conn:
    with conn.cursor() as cursor:
        # batch_size = 500
        # for i in range(0, len(data_number), batch_size):
        #     batch = data_number[i:i+batch_size]
        #     cursor.executemany(insert_sql, batch)
        #     conn.commit()
        cnt = cursor.executemany(insert_sql, data_number_2)  # executemany() 결과 행수 반환.
        conn.commit()
        

# print("insert된 총 행수:", cnt)
print("실행 완료") # 1000까지 입력이 되었다. 

[['노원구', '2022-03', 2, 2, 134], ['노원구', '2022-03', 2, 3, 502], ['노원구', '2022-03', 3, 1, 3], ['노원구', '2022-03', 3, 2, 858], ['노원구', '2022-03', 3, 3, 848]]
--------------------------------------------------
['노원구', '2022-03', 2, 1, 561]
--------------------------------------------------
['노원구', '2022-03', 2, 2, 134]
실행 완료


In [24]:
len(data_number_2)
data_number_2[999]

['성북구', '2022-06', 3, 2, 201]